In [1]:
import polars as pl
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 8),
            nn.ReLU(), 
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim) 
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [13]:
def startTrain(X, y, hidden_dim, epochs, learning_rate, batch_size):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for fold, (train_idx, test_idx) in enumerate(kfold.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Aplicar MinMaxScaler APENAS aos dados de treino
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Filtrar apenas os dados benignos (classe 0) para treinar o Autoencoder
        X_train = X_train[y_train == 0]

        # Converter para tensores PyTorch
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

        # Criar DataLoaders
        train_dataset = TensorDataset(X_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Configuração do dispositivo e modelo
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        input_dim = X_train.shape[1]
        model = Autoencoder(input_dim=input_dim, hidden_dim=hidden_dim).to(device)

        # Definição da função de perda e otimizador
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Treinamento
        start_training = time.time()
        for epoch in range(epochs):
            model.train()
            epoch_loss = 0
            for data in train_loader:
                inputs = data[0].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, inputs)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)

        end_training = time.time()

        # Avaliação do limiar baseado no conjunto de treino
        model.eval()
        reconstruction_errors_train = []
        with torch.no_grad():
            for data in train_loader:
                inputs = data[0].to(device)
                outputs = model(inputs)
                reconstruction_error = torch.mean((outputs - inputs) ** 2, dim=1)
                reconstruction_errors_train.extend(reconstruction_error.cpu().numpy())

        reconstruction_errors_train = np.array(reconstruction_errors_train)
        threshold = np.percentile(reconstruction_errors_train, 95)  # Limiar baseado no percentil 95

        # Avaliação no conjunto de teste
        reconstruction_errors_test = []
        y_pred = []
        with torch.no_grad():
            for data in test_loader:
                inputs, targets = data[0].to(device), data[1].cpu().numpy()
                outputs = model(inputs)
                reconstruction_error = torch.mean((outputs - inputs) ** 2, dim=1).cpu().numpy()
                reconstruction_errors_test.extend(reconstruction_error)
                y_pred.extend((reconstruction_error > threshold).astype(int))

        # Conversão para numpy arrays
        reconstruction_errors_test = np.array(reconstruction_errors_test)
        y_pred = np.array(y_pred)

        # Cálculo de métricas de desempenho
        evaluation_time = time.time()
        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0

        training_duration = end_training - start_training
        evaluation_duration = time.time() - end_training
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["AE"] + mean_results.tolist())

In [14]:
#Melhores Hiperparâmetros encontrados: {'hidden_dim': 16, 'epochs': 20, 'learning_rate': 0.001, 'batch_size': 512}

In [15]:
for i in range(1,11):
    startTrain(X, y, hidden_dim=16, epochs=20, learning_rate=0.001, batch_size=512)
    print(i)

1
2
3
4
5
6
7
8
9
10


In [16]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy' , 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""AE""",0.98327,0.969887,0.990481,0.989586,0.950187,0.990033,0.049813,16916.400391,886.799988,971.0,92272.398438,10.863027,1.430868
"""AE""",0.983643,0.970066,0.990464,0.990052,0.95008,0.990257,0.049921,16914.400391,888.799988,927.599976,92315.796875,11.031323,1.341935
"""AE""",0.981685,0.968815,0.990404,0.987758,0.949872,0.989079,0.050128,16910.800781,892.400024,1141.400024,92102.0,10.211519,1.324315
"""AE""",0.98293,0.969616,0.990445,0.989213,0.950019,0.989828,0.049981,16913.400391,889.799988,1005.799988,92237.601562,10.469568,1.347683
"""AE""",0.982487,0.969315,0.990423,0.988705,0.949925,0.989561,0.050075,16911.800781,891.400024,1053.400024,92190.0,10.771115,1.402249
"""AE""",0.983567,0.970001,0.990455,0.989971,0.950032,0.990212,0.049968,16913.599609,889.599976,935.200012,92308.203125,10.816902,1.431598
"""AE""",0.981655,0.968914,0.990458,0.987668,0.950159,0.989059,0.049841,16916.0,887.200012,1150.0,92093.398438,13.489792,1.67585
"""AE""",0.97918,0.967453,0.990434,0.984721,0.950185,0.987554,0.049815,16916.400391,886.799988,1425.199951,91818.203125,12.011002,1.58137
"""AE""",0.978359,0.96713,0.990502,0.983653,0.950608,0.987043,0.049392,16923.800781,879.400024,1523.800049,91719.601562,12.33753,1.598603


In [17]:
metrics_df.write_csv(f"metrics_results/unbalanced_AE_metrics_output.csv", separator=';')